In [17]:
import os
import torch


tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}
SMOKE_TEST = os.environ.get("SMOKE_TEST")

### Problem setup


In [18]:

def f_1(x):
    r = 1/x[:,0]+ x[:,1]
    return(-r.unsqueeze(1))

def f_2(x):
    r = x[:,0]+ x[:,1]**2
    return(-r.unsqueeze(1))

def problem(X):
    return torch.cat([f_1(X), f_2(X)], dim = -1)
bounds = torch.tensor([[1]*2, [1.5]*2]).to( **tkwargs)


d = 2
M = 2

In [19]:
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from botorch.utils.sampling import draw_sobol_samples
from botorch.utils.transforms import normalize, unnormalize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from gpytorch.kernels import RBFKernel, ScaleKernel
NOISE_SE = torch.tensor([0.1]*M).to(**tkwargs) 
def generate_initial_data(n):
    # generate training data
    train_x = draw_sobol_samples(bounds=bounds, n=n, q=1).squeeze(1)
    train_obj_true = problem(train_x)
    train_obj = train_obj_true + torch.randn_like(train_obj_true) * NOISE_SE
    # negative values imply feasibility in botorch
    # train_con = -evaluate_slack(train_x)
    return train_x, train_obj, train_obj_true

base = RBFKernel()
covar_module = ScaleKernel(
base_kernel=base,
)
# def initialize_model(train_x, train_obj, train_con):
#     # define models for objective and constraint
#     train_x = normalize(train_x, bounds)
#     train_y = torch.cat([train_obj, train_con], dim=-1)
#     models = []
#     for i in range(train_y.shape[-1]):
#         models.append(
#             SingleTaskGP(
#                 train_x, train_y[..., i : i + 1], outcome_transform=Standardize(m=1),train_Yvar= torch.zeros((train_x.shape[0],1)) + 0.01**2,covar_module = covar_module)
#             )
        
#     model = ModelListGP(*models)
#     mll = SumMarginalLogLikelihood(model.likelihood, model)
#     return mll, model

def initialize_model(train_x, train_obj):
    # define models for objective and constraint
    train_x = normalize(train_x, bounds)
    models = []
    for i in range(train_obj.shape[-1]):
        train_y = train_obj[..., i : i + 1]
        train_yvar = torch.full_like(train_y, NOISE_SE[i] ** 2)
        models.append(
            SingleTaskGP(
                train_x, train_y, outcome_transform=Standardize(m=1),train_Yvar= torch.zeros((train_x.shape[0],1)) + 0.01,covar_module = covar_module
            )
        )
    model = ModelListGP(*models)
    mll = SumMarginalLogLikelihood(model.likelihood, model)
    return mll, model

In [20]:
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.utils.sampling import sample_simplex


BATCH_SIZE = 1
NUM_RESTARTS = 10 if not SMOKE_TEST else 2
RAW_SAMPLES = 512 if not SMOKE_TEST else 4

standard_bounds = torch.zeros(2, d, **tkwargs)
standard_bounds[1] = 1


def optimize_qehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, bounds)).mean
    partitioning = FastNondominatedPartitioning(
        ref_point=torch.tensor([-3.0,-4.0]),
        Y=pred,
    )
    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=torch.tensor([-3.0,-4.0]),
        partitioning=partitioning,
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
    # observe new values
    new_x = unnormalize(candidates.detach(), bounds=bounds)
    new_obj_true = problem(new_x)
    new_obj = new_obj_true + torch.randn_like(new_obj_true) *NOISE_SE
    return new_x, new_obj, new_obj_true

In [21]:
def optimize_qnehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=torch.tensor([-3.0,-4.0]).tolist(),  # use known reference point
        X_baseline=normalize(train_x, bounds),
        prune_baseline=True,  # prune baseline points that have estimated zero probability of being Pareto optimal
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
        sequential=True,
    )
    # observe new values
    new_x = unnormalize(candidates.detach(), bounds=bounds)
    new_obj_true = problem(new_x)
    new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj, new_obj_true

In [22]:
from botorch.acquisition.monte_carlo import qNoisyExpectedImprovement


def optimize_qnparego_and_get_observation(model, train_x, train_obj, sampler):
    """Samples a set of random weights for each candidate in the batch, performs sequential greedy optimization
    of the qNParEGO acquisition function, and returns a new candidate and observation."""
    train_x = normalize(train_x, bounds)
    with torch.no_grad():
        pred = model.posterior(train_x).mean
    acq_func_list = []
    for _ in range(BATCH_SIZE):
        weights = sample_simplex(problem.num_objectives, **tkwargs).squeeze()
        objective = GenericMCObjective(
            get_chebyshev_scalarization(weights=weights, Y=pred)
        )
        acq_func = qNoisyExpectedImprovement(  # pyre-ignore: [28]
            model=model,
            objective=objective,
            X_baseline=train_x,
            sampler=sampler,
            prune_baseline=True,
        )
        acq_func_list.append(acq_func)
    # optimize
    candidates, _ = optimize_acqf_list(
        acq_function_list=acq_func_list,
        bounds=standard_bounds,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200},
    )
    # observe new values
    new_x = unnormalize(candidates.detach(), bounds=bounds)
    new_obj_true = problem(new_x)
    new_obj = new_obj_true + torch.randn_like(new_obj_true) * NOISE_SE
    return new_x, new_obj, new_obj_true

In [23]:

def voxel_grid_sampling_with_indices(points, voxel_size = 5.0):
    # Calculate the minimum and maximum coordinates
    min_coords = torch.min(points, dim=0).values
    max_coords = torch.max(points, dim=0).values

    # Shift points so that the minimum coordinates are at the origin
    shifted_points = points - min_coords

    # Quantize the points to voxel grid coordinates
    voxel_indices = torch.floor(shifted_points / voxel_size).long()

    # Use a dictionary to store unique voxel indices and the corresponding row index
    voxel_dict = {}
    for idx, voxel_idx in enumerate(voxel_indices):
        voxel_idx_tuple = tuple(voxel_idx.tolist())
        if voxel_idx_tuple not in voxel_dict:
            voxel_dict[voxel_idx_tuple] = idx

    # Extract the row indices of the sampled points
    sampled_indices = torch.tensor(list(voxel_dict.values()))

    return sampled_indices



In [ ]:
import time
import warnings

from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.dominated import (
    DominatedPartitioning,
)
from botorch.utils.multi_objective.pareto import is_non_dominated


warnings.filterwarnings('ignore')
random_seeds = [83810, 14592, 3278, 97196, 36048, 32098, 29256, 18289, 96530, 13434, 88696, 97080, 71482, 11395, 77397, 55302, 4165, 3905, 12280, 28657, 30495, 66237, 78907, 3478, 73563,
26062, 93850, 85181, 91924, 71426, 54987, 28893, 58878, 77236, 36463, 851, 99458, 20926, 91506, 55392, 44597, 36421, 20379, 28221, 44118, 13396, 12156, 49797, 12676, 47052]
declared = False

N_BATCH = 100
MC_SAMPLES = 128 if not SMOKE_TEST else 16
verbose = True
c = 0

verbose = True
for seed in random_seeds[:10]:
    torch.manual_seed(seed)
    train_x_qnehvi, train_obj_qnehvi, train_obj_true_qnehvi= generate_initial_data(10)
    # resample_ind = voxel_grid_sampling_with_indices(train_obj_qnehvi)
    # train_x_qnehvi = train_x_qnehvi[resample_ind, :]
    # train_obj_qnehvi = train_obj_qnehvi[resample_ind, :]
    # train_con_qnehvi = train_con_qnehvi[resample_ind, :]
    hvs_qnehvi = []

    # # call helper functions to generate initial training data and initialize model
    # train_x_qparego, train_obj_qparego, train_obj_true_qparego = generate_initial_data(
    #     n=64
    # )
    # mll_qparego, model_qparego = initialize_model(train_x_qparego, train_obj_qparego)

    # train_x_qehvi, train_obj_qehvi, train_obj_true_qehvi = (
    #     train_x_qparego,
    #     train_obj_qparego,
    #     train_obj_true_qparego,
    # )
    # train_x_qnehvi, train_obj_qnehvi, train_obj_true_qnehvi = (
    #     train_x_qparego,
    #     train_obj_qparego,
    #     train_obj_true_qparego,
    # )
    # train_x_random, train_obj_random, train_obj_true_random = (
    #     train_x_qparego,
    #     train_obj_qparego,
    #     train_obj_true_qparego,
    # )
    # mll_qehvi, model_qehvi = initialize_model(train_x_qehvi, train_obj_qehvi)
    mll_qnehvi, model_qnehvi = initialize_model(train_x_qnehvi, train_obj_qnehvi)

    # compute hypervolume
    bd = DominatedPartitioning(ref_point=torch.tensor([-1.9,-2.25]), Y=train_obj_true_qnehvi)
    volume = bd.compute_hypervolume().item()

    # hvs_qparego.append(volume)
    # hvs_qehvi.append(volume)
    hvs_qnehvi.append(volume)

    # run N_BATCH rounds of BayesOpt after the initial random batch
    for iteration in range(1, N_BATCH + 1):

        t0 = time.monotonic()

        # fit the models
        # fit_gpytorch_mll(mll_qparego)
        # fit_gpytorch_mll(mll_qehvi)
        fit_gpytorch_mll(mll_qnehvi)

        # define the qEI and qNEI acquisition modules using a QMC sampler
        # qparego_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
        # qehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))
        qnehvi_sampler = SobolQMCNormalSampler(sample_shape=torch.Size([MC_SAMPLES]))

        # # optimize acquisition functions and get new observations
        # (
        #     new_x_qparego,
        #     new_obj_qparego,
        #     new_obj_true_qparego,
        # ) = optimize_qnparego_and_get_observation(
        #     model_qparego, train_x_qparego, train_obj_qparego, qparego_sampler
        # )
        # new_x_qehvi, new_obj_qehvi, new_obj_true_qehvi = optimize_qehvi_and_get_observation(
        #     model_qehvi, train_x_qehvi, train_obj_qehvi, qehvi_sampler
        # )
        (
            new_x_qnehvi,
            new_obj_qnehvi,
            new_obj_true_qnehvi,
        ) = optimize_qnehvi_and_get_observation(
            model_qnehvi, train_x_qnehvi, train_obj_qnehvi, qnehvi_sampler
        )
        # new_x_random, new_obj_random, new_obj_true_random = generate_initial_data(
        #     n=BATCH_SIZE
        # )

        # update training points
        # train_x_qparego = torch.cat([train_x_qparego, new_x_qparego])
        # train_obj_qparego = torch.cat([train_obj_qparego, new_obj_qparego])
        # train_obj_true_qparego = torch.cat([train_obj_true_qparego, new_obj_true_qparego])

        # train_x_qehvi = torch.cat([train_x_qehvi, new_x_qehvi])
        # train_obj_qehvi = torch.cat([train_obj_qehvi, new_obj_qehvi])
        # train_obj_true_qehvi = torch.cat([train_obj_true_qehvi, new_obj_true_qehvi])

        train_x_qnehvi = torch.cat([train_x_qnehvi, new_x_qnehvi])
        train_obj_qnehvi = torch.cat([train_obj_qnehvi, new_obj_qnehvi])
        train_obj_true_qnehvi = torch.cat([train_obj_true_qnehvi, new_obj_true_qnehvi])

        # train_x_random = torch.cat([train_x_random, new_x_random])
        # train_obj_random = torch.cat([train_obj_random, new_obj_random])
        # train_obj_true_random = torch.cat([train_obj_true_random, new_obj_true_random])

        # update progress
        bd = DominatedPartitioning(ref_point=torch.tensor([-1.9,-2.25]), Y=train_obj_true_qnehvi)
        volume = bd.compute_hypervolume().item()
        hvs_qnehvi.append(volume)

        # reinitialize the models so they are ready for fitting on next iteration
        # Note: we find improved performance from not warm starting the model hyperparameters
        # using the hyperparameters from the previous iteration
        # mll_qparego, model_qparego = initialize_model(train_x_qparego, train_obj_qparego)
        # mll_qehvi, model_qehvi = initialize_model(train_x_qehvi, train_obj_qehvi)
        mll_qnehvi, model_qnehvi = initialize_model(train_x_qnehvi, train_obj_qnehvi)

        t1 = time.monotonic()
        if verbose:
            print(
                f"({hvs_qnehvi[-1]:>4.2f}), "
                f"time = {t1-t0:>4.2f}.",
                end="",
            )
        else:
            print(".", end="")
    c+=1
    torch.save( hvs_qnehvi, f'toy_unconstrained_hv_{c}.pt')
    torch.save(train_obj_true_qnehvi, f'toy_unconstrained_obj_true_{c}.pt')
    torch.save(train_obj_qnehvi, f'toy_unconstrained_obj_{c}.pt')